In [ ]:

import cv2				 # working with, mainly resizing, images
import numpy as np		 # dealing with arrays
import os				  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm	  # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf

trainDir = '../input/train/'
testDir = '../input/test'
#testDir = '../input/train/'
LR = 1e-3

imageSize = 50
#MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match

def label_img(img):
	word_label = img.split('.')[-3]
	# conversion to one-hot array [cat,dog]
	#							[much cat, no dog]
	if word_label == 'cat': return [1,0]
	#							 [no cat, very doggo]
	elif word_label == 'dog': return [0,1]
	
	
def create_train_data():
	training_data = []
	for img in os.listdir(trainDir):
		label = label_img(img)
		path = os.path.join(trainDir,img)
		img = cv2.imread(path)
		img = cv2.resize(img, (imageSize,imageSize))
		training_data.append([np.array(img),np.array(label), ])
	shuffle(training_data)
	#np.save('train_data.npy', training_data)
	return training_data
	
	
def process_test_data():
	testing_data = []
	for img in os.listdir(testDir):
		path = os.path.join(testDir,img)
		img_num = img.split('.')[0]
		img = cv2.imread(path)
		img = cv2.resize(img, (imageSize,imageSize))
		testing_data.append([np.array(img), img_num])
		
	shuffle(testing_data)
	#np.save('test_data.npy', testing_data)
	return testing_data
	
train_data = create_train_data()

testDataCount = int(len(train_data) / 5)

testData = train_data[:testDataCount]
train_data = train_data[testDataCount:]

convnet = input_data(shape=[None, imageSize, imageSize, 3], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

	
X = np.array([i[0] for i in train_data]).reshape(-1,imageSize,imageSize,3)
Y = [i[1] for i in train_data]

test_x = np.array([i[0] for i in testData]).reshape(-1,imageSize,imageSize,3)
test_y = [i[1] for i in testData]

model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
	snapshot_step=5000, show_metric=True)
testData = process_test_data()

for data in testData:
	img_num = data[1]
	img_data = data[0]
	orig = img_data
	data = img_data.reshape(imageSize,imageSize,3)
	model_out = model.predict([data])[0]
	print('{},{}\n'.format(img_num,model_out[1]))